1. ### 明天上午做的事情，来调参，借助grid，以及用xgboost做一下，用bayes做一下模型
### 想想为什么随机森林效果很差
### 作进一步的特征提取
### 可以直接在kernel上做啊

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('../input/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
df.to_csv('output.csv')

In [ ]:
y_train = df['Survived']  #训练集的label

In [9]:
'''
画图显示中文
'''
#-*- coding: utf-8 -*-
from matplotlib.font_manager import FontProperties
font = FontProperties(fname=r"c:\windows\fonts\simsun.ttc", size=14)

In [ ]:
df.info()

In [ ]:
#用图表分析一波
fig = plt.figure()
fig.set(alpha = 0.2)

plt.subplot2grid((2, 3), (0, 0))
df.Survived.value_counts().plot(kind = 'bar')
plt.ylabel('救/没救人数',fontproperties=font)
plt.title(u'救和没有获救对比', fontproperties=font)
plt.grid(True)

plt.subplot2grid((2, 3), (0, 1))
df.Pclass.value_counts().plot(kind = 'bar')
plt.ylabel('船舱等级',fontproperties=font)
plt.title('船舱', fontproperties=font)
plt.grid(True)

plt.subplot2grid((2, 3), (0, 2))
plt.scatter(df.Survived, df.Age)
plt.grid(True)
plt.title('年龄',fontproperties=font)

plt.subplot2grid((2, 3), (1, 0), colspan=2)  #占举两格
df['Age'][df['Pclass'] == 1].plot(kind = 'kde', label = 'p1')
df['Age'][df['Pclass'] == 2].plot(kind = 'kde', label = 'p2')
df['Age'][df['Pclass'] == 3].plot(kind = 'kde', label = 'p3')
plt.legend(loc = 'best')
plt.title(u'各舱年龄的密度图',  fontproperties=font)

plt.subplot2grid((2, 3), (1, 2))
df.Embarked.value_counts().plot(kind = 'bar')
plt.title('Embarked')
plt.grid(True)

In [ ]:
fig = plt.figure()
fig.set(alpha = 0.2)

Survived_0 = df.Pclass[df.Survived == 0].value_counts()
Survived_1 = df.Pclass[df.Survived == 1].value_counts()
plot_df = pd.DataFrame({'saved':Survived_1, 'Not saved':Survived_0})

plot_df.plot(kind = 'bar', stacked = True)
plt.xlabel(u'船舱等级', fontproperties = font)
plt.ylabel(u'人数', fontproperties = font)
plt.legend(loc = 'best')


In [ ]:
fig = plt.figure()
fig.set(alpha = 0.2)

Survived_0 = df.Sex[df.Survived == 0].value_counts()
Survived_1 = df.Sex[df.Survived == 1].value_counts()
plot_df = pd.DataFrame({'saved':Survived_1, 'Not saved':Survived_0})

plot_df.plot(kind = 'bar', stacked = True)
plt.xlabel(u'性别', fontproperties = font)
plt.ylabel(u'人数', fontproperties = font)
plt.legend(loc = 'best')


In [ ]:
fig = plt.figure()
fig.set(alpha = 0.2)
#plt.title(u'根据各舱和性别的对别分析', fontproperties = font)

ax1 = fig.add_subplot(141)
df.Survived[df.Sex == 'female'][df.Pclass != 3].value_counts().plot(kind = 'bar', label = u'女/高')
ax1.legend(loc = 'best',  prop =  font)
plt.ylim(ymax=500)

ax2 = fig.add_subplot(142)
df.Survived[df.Sex == 'female'][df.Pclass == 3].value_counts().plot(kind = 'bar', label = u'女/低')
ax2.legend(loc = 'best',  prop =  font)
plt.ylim(ymax=500)

ax3 = fig.add_subplot(143)
df.Survived[df.Sex == 'male'][df.Pclass != 3].value_counts().plot(kind = 'bar', label = u'男/高')
ax3.legend(loc = 'best',  prop =  font)
plt.ylim(ymax=500)

ax4 = fig.add_subplot(144)
df.Survived[df.Sex == 'male'][df.Pclass == 3].value_counts().plot(kind = 'bar', label = u'男/低')
ax4.legend(loc = 'best',  prop =  font)
plt.ylim(ymax=500)

In [ ]:
fig = plt.figure()
fig.set(alpha = 0.2)
#plt.title(u'根据各舱和性别的对别分析', fontproperties = font)

ax1 = fig.add_subplot(131)
df.Survived[df.Embarked == 'S'].value_counts().plot(kind = 'bar', label = u'S港口')
ax1.legend(loc = 'best',  prop =  font)
plt.ylim(ymax=500)

ax2 = fig.add_subplot(132)
df.Survived[df.Embarked == 'C'].value_counts().plot(kind = 'bar', label = u'C港口')
ax2.legend(loc = 'best',  prop =  font)
plt.ylim(ymax=500)

ax3 = fig.add_subplot(133)
df.Survived[df.Embarked == 'Q'].value_counts().plot(kind = 'bar', label = u'Q港口')
ax3.legend(loc = 'best',  prop =  font)
plt.ylim(ymax=500)

In [ ]:
fig = plt.figure()
fig.set(alpha = 0.2)

Cabin_null = df.Survived[df.Cabin.isnull()].value_counts()
Cabin_notnull = df.Survived[df.Cabin.notnull()].value_counts()
df_plot = pd.DataFrame({'没有标注信息':Cabin_null, '有标注信息':Cabin_notnull})

df_plot.plot(kind = 'bar', stacked = True)
plt.xlabel(u'生存与否', fontproperties = font)
plt.ylabel(u'人数', fontproperties = font)
plt.legend(loc = 'best', prop = font)


In [ ]:
df.head()

In [ ]:
g = pd.DataFrame(df.groupby(['Parch', 'Survived']).count())
h = pd.DataFrame(df.groupby(['SibSp', 'Survived']).count())

In [ ]:
df.Age[df.Survived == 0].mean()
df.Age[df.Survived == 1].mean()

In [10]:
df.head()
df_copy = df   #df是原始数据，不要打乱，新建备份来做

In [ ]:
df_copy.head()

In [11]:
#导入必要的包
from sklearn.preprocessing import StandardScaler

In [12]:
#分析完图表以后，开始做训练集
#先对Fare归一化
scaler = StandardScaler()
df_copy['fare_scaled'] = scaler.fit_transform(np.array(df_copy['Fare'].values.reshape(-1, 1)))
df_copy.drop('Fare',axis=1, inplace=True)

In [13]:
#先将Sex和Pclass构造一个组合特征
df_copy['Sex_Pclass'] = df_copy['Sex'] + '_' + df_copy['Pclass'].map(str)  #将性别和Pclass单独组合起来成为一个新的属性

In [14]:
df_copy.loc[ (df_copy.Cabin.notnull()), 'Cabin' ] = "Yes"
df_copy.loc[ (df_copy.Cabin.isnull()), 'Cabin' ] = "No"

In [15]:
df_copy.loc[ (df_copy.Embarked == 'S'), 'Embarked' ] = "S"
df_copy.loc[ (df_copy.Embarked != 'S'), 'Embarked' ] = "other"

In [16]:
df_copy.loc[(df_copy.SibSp != 0), 'SibSp'] = 1
df_copy.loc[(df_copy.SibSp == 0), 'SibSp'] = 0
df_copy.loc[(df_copy.Parch != 0), 'Parch'] = 1
df_copy.loc[(df_copy.Parch == 0), 'Parch'] = 0

In [17]:
#对于年龄来说，存在缺失值，稍后做
#先将Sex和Pclass转换成one-hot，并且构造一个组合特征
dummies_Cabin = pd.get_dummies(df_copy['Cabin'], prefix= 'Cabin')

dummies_Parch = pd.get_dummies(df_copy['Parch'], prefix= 'Parch')

dummies_SibSp = pd.get_dummies(df_copy['SibSp'], prefix= 'SibSp')

dummies_Embarked = pd.get_dummies(df_copy['Embarked'], prefix= 'Embarked')

dummies_Sex = pd.get_dummies(df_copy['Sex'], prefix= 'Sex')

dummies_Pclass = pd.get_dummies(df_copy['Pclass'], prefix= 'Pclass')

dummies_Sex_Pclass = pd.get_dummies(df_copy['Sex_Pclass'], prefix= 'Sex_Pclass')

df_copy = pd.concat([df_copy, dummies_Cabin, dummies_Parch, dummies_SibSp,\
                     dummies_Embarked, dummies_Sex, dummies_Pclass, dummies_Sex_Pclass], axis=1)
df_copy.drop(['Pclass', 'Sex', 'Cabin', 'Embarked', 'Sex_Pclass', 'Parch', 'SibSp'], axis=1, inplace=True)



In [18]:
df_copy.drop(['Name', 'Ticket'], axis = 1, inplace = True)

In [19]:
df_copy.head()

,PassengerId,Survived,Age,fare_scaled,Cabin_No,Cabin_Yes,Parch_0,Parch_1,SibSp_0,SibSp_1,...,Sex_male,Pclass_1,Pclass_2,Pclass_3,Sex_Pclass_female_1,Sex_Pclass_female_2,Sex_Pclass_female_3,Sex_Pclass_male_1,Sex_Pclass_male_2,Sex_Pclass_male_3
0,1,0,22.0,-0.502445,1,0,1,0,0,1,...,1,0,0,1,0,0,0,0,0,1
1,2,1,38.0,0.786845,0,1,1,0,0,1,...,0,1,0,0,1,0,0,0,0,0
2,3,1,26.0,-0.488854,1,0,1,0,1,0,...,0,0,0,1,0,0,1,0,0,0
3,4,1,35.0,0.420730,0,1,1,0,0,1,...,0,1,0,0,1,0,0,0,0,0
4,5,0,35.0,-0.486337,1,0,1,0,1,0,...,1,0,0,1,0,0,0,0,0,1


In [20]:
#age目前还没有解决
#用中位数填充缺失值
df_copy['Age_fill'] = df_copy.Age.fillna(df_copy.Age.mean())

In [21]:
scaler = StandardScaler()
df_copy['Age_fill_scaled'] = scaler.fit_transform(np.array(df_copy['Age_fill'].values.reshape(-1, 1)))
df_copy.drop(['Age_fill', 'Age'],axis=1, inplace=True)

In [ ]:
df_copy.head()

In [22]:
# 我们把需要的feature字段取出来，转成numpy格式，使用scikit-learn中的LogisticRegression建模
from sklearn import linear_model

train_df = df_copy.filter(regex='Survived|Age_.*|SibSp_.*|Parch_.*|fare_.*|Cabin_.*|Embarked_.*|Sex_.*|Pclass_.*')
train_np = train_df.as_matrix()

# y即Survival结果
y = train_np[:, 0]

# X即特征属性值
X = train_np[:, 1:]
    

In [ ]:
# fit到RandomForestRegressor之中
clf = linear_model.LogisticRegression(C=1.0, penalty='l2', tol=1e-4)
clf.fit(X, y)

In [ ]:
pd.DataFrame({"columns":list(train_df.columns)[1:], "coef":list(clf.coef_.T)})

In [ ]:
df_test = pd.read_csv('data/test.csv')
df_test.head()

In [ ]:
df_test.info()

In [24]:
#对测试集做同样的事情
#分析完图表以后，开始做训练集
#先对Fare归一化
df_test = pd.read_csv('../input/test.csv')

scaler = StandardScaler()
df_test['Fare'] = df_test.Fare.fillna(df_test.Fare.mean())
df_test['fare_scaled'] = scaler.fit_transform(np.array(df_test['Fare'].values.reshape(-1, 1)))
df_test.drop('Fare',axis=1, inplace=True)

#先将Sex和Pclass构造一个组合特征
df_test['Sex_Pclass'] = df_test['Sex'] + '_' + df_test['Pclass'].map(str)  #将性别和Pclass单独组合起来成为一个新的属性

df_test.loc[ (df_test.Cabin.notnull()), 'Cabin' ] = "Yes"
df_test.loc[ (df_test.Cabin.isnull()), 'Cabin' ] = "No"

df_test.loc[ (df_test.Embarked == 'S'), 'Embarked' ] = "S"
df_test.loc[ (df_test.Embarked != 'S'), 'Embarked' ] = "other"

df_test.loc[(df_test.SibSp != 0), 'SibSp'] = 1
df_test.loc[(df_test.SibSp == 0), 'SibSp'] = 0
df_test.loc[(df_test.Parch != 0), 'Parch'] = 1
df_test.loc[(df_test.Parch == 0), 'Parch'] = 0

#对于年龄来说，存在缺失值，稍后做
#先将Sex和Pclass转换成one-hot，并且构造一个组合特征
dummies_Cabin = pd.get_dummies(df_test['Cabin'], prefix= 'Cabin')

dummies_Parch = pd.get_dummies(df_test['Parch'], prefix= 'Parch')

dummies_SibSp = pd.get_dummies(df_test['SibSp'], prefix= 'SibSp')

dummies_Embarked = pd.get_dummies(df_test['Embarked'], prefix= 'Embarked')

dummies_Sex = pd.get_dummies(df_test['Sex'], prefix= 'Sex')

dummies_Pclass = pd.get_dummies(df_test['Pclass'], prefix= 'Pclass')

dummies_Sex_Pclass = pd.get_dummies(df_test['Sex_Pclass'], prefix= 'Sex_Pclass')

df_test = pd.concat([df_test, dummies_Cabin, dummies_Parch, dummies_SibSp,\
                     dummies_Embarked, dummies_Sex, dummies_Pclass, dummies_Sex_Pclass], axis=1)
df_test.drop(['Pclass', 'Sex', 'Cabin', 'Embarked', 'Sex_Pclass', 'Parch', 'SibSp'], axis=1, inplace=True)

df_test.drop(['Name', 'Ticket'], axis = 1, inplace = True)

#age目前还没有解决
#用中位数填充缺失值
df_test['Age_fill'] = df_test.Age.fillna(df_test.Age.mean())

scaler = StandardScaler()
df_test['Age_fill_scaled'] = scaler.fit_transform(np.array(df_test['Age_fill'].values.reshape(-1, 1)))
df_test.drop(['Age_fill', 'Age'],axis=1, inplace=True)


In [25]:
test_df = df_test.filter(regex='Age_.*|SibSp_.*|Parch_.*|fare_.*|Cabin_.*|Embarked_.*|Sex_.*|Pclass_.*')
test_np = test_df.as_matrix()

In [ ]:
#预测值
predict_test = clf.predict(test_np)

In [ ]:
predict_test = clf.predict(test_np)
result = pd.DataFrame({'PassengerId':df_test['PassengerId'].as_matrix(), 'Survived':predict_test.astype(np.int32)})
#写成标准的数据上传格式

result.to_csv("logistic_regression_predictions.csv", index=False)  #

In [ ]:
from sklearn.learning_curve import learning_curve

# 用sklearn的learning_curve得到training_score和cv_score，使用matplotlib画出learning curve
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None, n_jobs=1, 
                        train_sizes=np.linspace(.05, 1., 20), verbose=0, plot=True):
    """
    画出data在某模型上的learning curve.
    参数解释
    ----------
    estimator : 你用的分类器。
    title : 表格的标题。
    X : 输入的feature，numpy类型
    y : 输入的target vector
    ylim : tuple格式的(ymin, ymax), 设定图像中纵坐标的最低点和最高点
    cv : 做cross-validation的时候，数据分成的份数，其中一份作为cv集，其余n-1份作为training(默认为3份)
    n_jobs : 并行的的任务数(默认1)
    """
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, verbose=verbose)
    
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    
    if plot:
        plt.figure()
        plt.title(title)
        if ylim is not None:
            plt.ylim(*ylim)
        plt.xlabel(u"训练样本数")
        plt.ylabel(u"得分")
        plt.gca().invert_yaxis()
        plt.grid()
    
        plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, 
                         alpha=0.1, color="b")
        plt.fill_between(train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, 
                         alpha=0.1, color="r")
        plt.plot(train_sizes, train_scores_mean, 'o-', color="b", label=u"训练集上得分")
        plt.plot(train_sizes, test_scores_mean, 'o-', color="r", label=u"交叉验证集上得分")
    
        plt.legend(loc="best")
        
        plt.draw()
        plt.gca().invert_yaxis()
        plt.show()
    
    midpoint = ((train_scores_mean[-1] + train_scores_std[-1]) + (test_scores_mean[-1] - test_scores_std[-1])) / 2
    diff = (train_scores_mean[-1] + train_scores_std[-1]) - (test_scores_mean[-1] - test_scores_std[-1])
    return midpoint, diff

plot_learning_curve(clf, u"学习曲线", X, y)

In [ ]:
from sklearn import linear_model
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingRegressor, BaggingClassifier, RandomForestClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor


#clf = linear_model.LogisticRegression(penalty = 'l2', tol = 1e-4)
#clf = RandomForestRegressor(n_estimators=100, random_state = 0, max_depth = None)
#clf = svm.SVC()
clf = DecisionTreeClassifier()
#clf = BaggingRegressor(clf, n_estimators=20, max_samples=0.8)
    
#trainModel = clf.fit(X, y)
#cross_val_score(clf, X, y, cv=5)
clf.fit(X, y)
#clf.score(X, y)

In [ ]:
print(cross_val_score(clf, X, y, cv=5))

In [ ]:
clf.score(X, y)

In [ ]:
predict_test = clf.predict(test_np)
result = pd.DataFrame({'PassengerId':df_test['PassengerId'].as_matrix(), 'Survived':predict_test.astype(np.int32)})
#写成标准的数据上传格式

result.to_csv("pred_label/DT_predictions.csv", index=False)  #

In [ ]:
result

In [28]:
#用xgboost来做一下
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.grid_search import GridSearchCV

In [42]:
#test_results = pd.read_csv('test_results.csv')
def modelfit(alg, train_x, train_y):

    X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.3, random_state=42)
    #建模
    alg.fit(X_train,y_train ,eval_metric='auc')
    print(X_train.shape, X_test.shape)
    #对训练集预测
    dtrain_predictions = alg.predict(X_test)
    dtrain_predprob = alg.predict_proba(X_test)[:,1]
        
    #输出模型的一些结果
    print("\n关于现在这个模型")
    print ("准确率 : %.4g" % metrics.accuracy_score(y_test, dtrain_predictions))
    print ("AUC 得分 (训练集): %f" % metrics.roc_auc_score(y_test, dtrain_predprob))
                
    #feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    #feat_imp.plot(kind='bar', title='Feature Importances')
    #plt.ylabel('Feature Importance Score')

In [43]:
xgb1 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=1000,
        max_depth=5,
        min_child_weight=1,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=1,
        scale_pos_weight=1,
        seed=27)
modelfit(xgb1, X, y)

(623, 21) (268, 21)

关于现在这个模型
准确率 : 0.7575
AUC 得分 (训练集): 0.826734


/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [45]:
#对subsample 和 max_features 用grid search查找最好的参数
param_test1 = {
    'max_depth':list(range(3,10,2)),
    'min_child_weight':list(range(1,6,2))
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
                                        min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=1, scale_pos_weight=1, seed=27), 
                       param_grid = param_test1, scoring='roc_auc',n_jobs=1,iid=False, cv=5)
gsearch1.fit(X,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=140,
       n_jobs=1, nthread=2, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=True,
       subsample=0.8),
       fit_params={}, iid=False, n_jobs=2,
       param_grid={'max_depth': [3, 5, 7, 9], 'min_child_weight': [1, 3, 5]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [46]:
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: 0.87095, std: 0.02854, params: {'max_depth': 3, 'min_child_weight': 1},
  mean: 0.87054, std: 0.02686, params: {'max_depth': 3, 'min_child_weight': 3},
  mean: 0.87068, std: 0.02684, params: {'max_depth': 3, 'min_child_weight': 5},
  mean: 0.87684, std: 0.03121, params: {'max_depth': 5, 'min_child_weight': 1},
  mean: 0.87571, std: 0.02645, params: {'max_depth': 5, 'min_child_weight': 3},
  mean: 0.87745, std: 0.02663, params: {'max_depth': 5, 'min_child_weight': 5},
  mean: 0.87380, std: 0.02906, params: {'max_depth': 7, 'min_child_weight': 1},
  mean: 0.87736, std: 0.02694, params: {'max_depth': 7, 'min_child_weight': 3},
  mean: 0.87814, std: 0.02486, params: {'max_depth': 7, 'min_child_weight': 5},
  mean: 0.87083, std: 0.02912, params: {'max_depth': 9, 'min_child_weight': 1},
  mean: 0.87627, std: 0.02676, params: {'max_depth': 9, 'min_child_weight': 3},
  mean: 0.87777, std: 0.02555, params: {'max_depth': 9, 'min_child_weight': 5}],
 {'max_depth': 7, 'min_child_weight': 5

In [49]:
param_test1 = {
    'max_depth':[4,5,6,7,8],
    'min_child_weight':[4,5,6]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=7,
                                        min_child_weight=5, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=1, scale_pos_weight=1, seed=27), 
                       param_grid = param_test1, scoring='roc_auc',n_jobs=1,iid=False, cv=5)
gsearch1.fit(X,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=7, min_child_weight=5, missing=None, n_estimators=140,
       n_jobs=1, nthread=1, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=True,
       subsample=0.8),
       fit_params={}, iid=False, n_jobs=1,
       param_grid={'max_depth': [4, 5, 6, 7, 8], 'min_child_weight': [4, 5, 6]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [50]:
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: 0.87502, std: 0.02626, params: {'max_depth': 4, 'min_child_weight': 4},
  mean: 0.87609, std: 0.02632, params: {'max_depth': 4, 'min_child_weight': 5},
  mean: 0.87518, std: 0.02666, params: {'max_depth': 4, 'min_child_weight': 6},
  mean: 0.87590, std: 0.02483, params: {'max_depth': 5, 'min_child_weight': 4},
  mean: 0.87745, std: 0.02663, params: {'max_depth': 5, 'min_child_weight': 5},
  mean: 0.87630, std: 0.02595, params: {'max_depth': 5, 'min_child_weight': 6},
  mean: 0.87676, std: 0.02494, params: {'max_depth': 6, 'min_child_weight': 4},
  mean: 0.87594, std: 0.02667, params: {'max_depth': 6, 'min_child_weight': 5},
  mean: 0.87837, std: 0.02552, params: {'max_depth': 6, 'min_child_weight': 6},
  mean: 0.87755, std: 0.02530, params: {'max_depth': 7, 'min_child_weight': 4},
  mean: 0.87814, std: 0.02486, params: {'max_depth': 7, 'min_child_weight': 5},
  mean: 0.87737, std: 0.02692, params: {'max_depth': 7, 'min_child_weight': 6},
  mean: 0.87669, std: 0.02424, params: {

In [51]:
param_test1 = {
    'min_child_weight':[5,6,7,8,10]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=8,
                                        min_child_weight=5, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=1, scale_pos_weight=1, seed=27), 
                       param_grid = param_test1, scoring='roc_auc',n_jobs=1,iid=False, cv=5)
gsearch1.fit(X,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=8, min_child_weight=5, missing=None, n_estimators=140,
       n_jobs=1, nthread=1, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=True,
       subsample=0.8),
       fit_params={}, iid=False, n_jobs=1,
       param_grid={'min_child_weight': [5, 6, 7, 8, 10]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [52]:
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: 0.87854, std: 0.02594, params: {'min_child_weight': 5},
  mean: 0.87712, std: 0.02596, params: {'min_child_weight': 6},
  mean: 0.87701, std: 0.02558, params: {'min_child_weight': 7},
  mean: 0.87433, std: 0.02542, params: {'min_child_weight': 8},
  mean: 0.86903, std: 0.02478, params: {'min_child_weight': 10}],
 {'min_child_weight': 5},
 0.8785422101538153)

In [53]:
#Grid seach选择合适的gamma
param_test3 = {
    'gamma':[i/10.0 for i in range(0,5)]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=8,
                                        min_child_weight=5, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=1, scale_pos_weight=1, seed=27), 
                       param_grid = param_test3, scoring='roc_auc',n_jobs=1,iid=False, cv=5)
gsearch1.fit(X,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=8, min_child_weight=5, missing=None, n_estimators=140,
       n_jobs=1, nthread=1, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=True,
       subsample=0.8),
       fit_params={}, iid=False, n_jobs=1,
       param_grid={'gamma': [0.0, 0.1, 0.2, 0.3, 0.4]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [54]:
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: 0.87854, std: 0.02594, params: {'gamma': 0.0},
  mean: 0.87790, std: 0.02535, params: {'gamma': 0.1},
  mean: 0.87797, std: 0.02538, params: {'gamma': 0.2},
  mean: 0.87880, std: 0.02534, params: {'gamma': 0.3},
  mean: 0.87716, std: 0.02451, params: {'gamma': 0.4}],
 {'gamma': 0.3},
 0.8788041789584529)

In [57]:
#Grid seach选择合适的gamma
param_test3 = {
    'gamma':[i/10.0 for i in range(0,5)]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=8,
                                        min_child_weight=5, gamma=0.3, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=1, scale_pos_weight=1, seed=27), 
                       param_grid = param_test3, scoring='accuracy',n_jobs=1,iid=False, cv=5)
gsearch1.fit(X,y)

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.3, learning_rate=0.1,
       max_delta_step=0, max_depth=8, min_child_weight=5, missing=None,
       n_estimators=140, n_jobs=1, nthread=1, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=27, silent=True, subsample=0.8),
       fit_params={}, iid=False, n_jobs=1,
       param_grid={'gamma': [0.0, 0.1, 0.2, 0.3, 0.4]},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [58]:
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: 0.83952, std: 0.01953, params: {'gamma': 0.0},
  mean: 0.83504, std: 0.01736, params: {'gamma': 0.1},
  mean: 0.84178, std: 0.02147, params: {'gamma': 0.2},
  mean: 0.83730, std: 0.02559, params: {'gamma': 0.3},
  mean: 0.83953, std: 0.01975, params: {'gamma': 0.4}],
 {'gamma': 0.2},
 0.8417846099723135)

In [59]:
xgb2 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=1000,
        max_depth=8,
        min_child_weight=5,
        gamma=0.2,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=1,
        scale_pos_weight=1,
        seed=27)
modelfit(xgb2, X, y)

(623, 21) (268, 21)

关于现在这个模型
准确率 : 0.791
AUC 得分 (训练集): 0.840908


/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [60]:
#Grid seach选择合适的gamma
param_test4 = {
    'n_estimators':[100, 200, 300,400,500,600,700,800,900,1000]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=8,
                                        min_child_weight=5, gamma=0.3, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=1, scale_pos_weight=1, seed=27), 
                       param_grid = param_test4, scoring='accuracy',n_jobs=1,iid=False, cv=5)
gsearch1.fit(X,y)

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.3, learning_rate=0.1,
       max_delta_step=0, max_depth=8, min_child_weight=5, missing=None,
       n_estimators=140, n_jobs=1, nthread=1, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=27, silent=True, subsample=0.8),
       fit_params={}, iid=False, n_jobs=1,
       param_grid={'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [61]:
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: 0.83618, std: 0.01751, params: {'n_estimators': 100},
  mean: 0.84178, std: 0.02265, params: {'n_estimators': 200},
  mean: 0.84512, std: 0.01755, params: {'n_estimators': 300},
  mean: 0.84738, std: 0.02409, params: {'n_estimators': 400},
  mean: 0.84629, std: 0.02334, params: {'n_estimators': 500},
  mean: 0.84627, std: 0.02457, params: {'n_estimators': 600},
  mean: 0.84179, std: 0.02542, params: {'n_estimators': 700},
  mean: 0.83729, std: 0.02164, params: {'n_estimators': 800},
  mean: 0.83281, std: 0.02332, params: {'n_estimators': 900},
  mean: 0.83394, std: 0.02225, params: {'n_estimators': 1000}],
 {'n_estimators': 400},
 0.8473771955115759)

In [62]:
#对subsample 和 colsample_bytree用grid search寻找最合适的参数
param_test4 = {
    'subsample':[i/10.0 for i in range(6,10)],
    'colsample_bytree':[i/10.0 for i in range(6,10)]
}

gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=400, max_depth=8,
                                        min_child_weight=5, gamma=0.3, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=1, scale_pos_weight=1, seed=27), 
                       param_grid = param_test4, scoring='roc_auc',n_jobs=1,iid=False, cv=5)
gsearch1.fit(X,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.3, learning_rate=0.1,
       max_delta_step=0, max_depth=8, min_child_weight=5, missing=None,
       n_estimators=400, n_jobs=1, nthread=1, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=27, silent=True, subsample=0.8),
       fit_params={}, iid=False, n_jobs=1,
       param_grid={'subsample': [0.6, 0.7, 0.8, 0.9], 'colsample_bytree': [0.6, 0.7, 0.8, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [63]:
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: 0.87756, std: 0.02726, params: {'colsample_bytree': 0.6, 'subsample': 0.6},
  mean: 0.87655, std: 0.02495, params: {'colsample_bytree': 0.6, 'subsample': 0.7},
  mean: 0.87911, std: 0.02796, params: {'colsample_bytree': 0.6, 'subsample': 0.8},
  mean: 0.87977, std: 0.02625, params: {'colsample_bytree': 0.6, 'subsample': 0.9},
  mean: 0.87749, std: 0.02780, params: {'colsample_bytree': 0.7, 'subsample': 0.6},
  mean: 0.88003, std: 0.02384, params: {'colsample_bytree': 0.7, 'subsample': 0.7},
  mean: 0.87866, std: 0.02542, params: {'colsample_bytree': 0.7, 'subsample': 0.8},
  mean: 0.87873, std: 0.02570, params: {'colsample_bytree': 0.7, 'subsample': 0.9},
  mean: 0.87700, std: 0.02656, params: {'colsample_bytree': 0.8, 'subsample': 0.6},
  mean: 0.88085, std: 0.02448, params: {'colsample_bytree': 0.8, 'subsample': 0.7},
  mean: 0.87836, std: 0.02396, params: {'colsample_bytree': 0.8, 'subsample': 0.8},
  mean: 0.87928, std: 0.02531, params: {'colsample_bytree': 0.8, 'subsample'

In [64]:
#对subsample 和 colsample_bytree用grid search寻找最合适的参数
param_test4 = {
    'subsample':[i/100.0 for i in range(65,80,5)],
    'colsample_bytree':[i/100.0 for i in range(75,90,5)]
}

gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=400, max_depth=8,
                                        min_child_weight=5, gamma=0.3, subsample=0.7, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=1, scale_pos_weight=1, seed=27), 
                       param_grid = param_test4, scoring='roc_auc',n_jobs=1,iid=False, cv=5)
gsearch1.fit(X,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.3, learning_rate=0.1,
       max_delta_step=0, max_depth=8, min_child_weight=5, missing=None,
       n_estimators=400, n_jobs=1, nthread=1, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=27, silent=True, subsample=0.7),
       fit_params={}, iid=False, n_jobs=1,
       param_grid={'subsample': [0.65, 0.7, 0.75], 'colsample_bytree': [0.75, 0.8, 0.85]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [65]:
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: 0.88066, std: 0.02508, params: {'colsample_bytree': 0.75, 'subsample': 0.65},
  mean: 0.87988, std: 0.02683, params: {'colsample_bytree': 0.75, 'subsample': 0.7},
  mean: 0.87957, std: 0.02592, params: {'colsample_bytree': 0.75, 'subsample': 0.75},
  mean: 0.88075, std: 0.02545, params: {'colsample_bytree': 0.8, 'subsample': 0.65},
  mean: 0.88085, std: 0.02448, params: {'colsample_bytree': 0.8, 'subsample': 0.7},
  mean: 0.87921, std: 0.02629, params: {'colsample_bytree': 0.8, 'subsample': 0.75},
  mean: 0.88000, std: 0.02648, params: {'colsample_bytree': 0.85, 'subsample': 0.65},
  mean: 0.87756, std: 0.02487, params: {'colsample_bytree': 0.85, 'subsample': 0.7},
  mean: 0.88084, std: 0.02765, params: {'colsample_bytree': 0.85, 'subsample': 0.75}],
 {'colsample_bytree': 0.8, 'subsample': 0.7},
 0.8808546479838626)

In [66]:
#对reg_alpha用grid search寻找最合适的参数
param_test6 = {
    'reg_alpha':[1e-5, 1e-2, 0.1, 0.5, 1]
}

gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=400, max_depth=8,
                                        min_child_weight=5, gamma=0.3, subsample=0.7, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=1, scale_pos_weight=1, seed=27), 
                       param_grid = param_test6, scoring='roc_auc',n_jobs=1,iid=False, cv=5)
gsearch1.fit(X,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.3, learning_rate=0.1,
       max_delta_step=0, max_depth=8, min_child_weight=5, missing=None,
       n_estimators=400, n_jobs=1, nthread=1, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=27, silent=True, subsample=0.7),
       fit_params={}, iid=False, n_jobs=1,
       param_grid={'reg_alpha': [1e-05, 0.01, 0.1, 0.5, 1]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [67]:
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: 0.88085, std: 0.02448, params: {'reg_alpha': 1e-05},
  mean: 0.88071, std: 0.02531, params: {'reg_alpha': 0.01},
  mean: 0.87956, std: 0.02662, params: {'reg_alpha': 0.1},
  mean: 0.87954, std: 0.02588, params: {'reg_alpha': 0.5},
  mean: 0.87990, std: 0.02460, params: {'reg_alpha': 1}],
 {'reg_alpha': 1e-05},
 0.8808546479838626)

In [68]:
xgb4 = XGBClassifier(
        learning_rate =0.01,
        n_estimators=5000,
        max_depth=8,
        min_child_weight=5,
        gamma=0.2,
        subsample=0.7,
        colsample_bytree=0.8,
        reg_alpha=1e-05,
        objective= 'binary:logistic',
        nthread=1,
        scale_pos_weight=1,
        seed=27)
modelfit(xgb4, X, y)

(623, 21) (268, 21)

关于现在这个模型
准确率 : 0.806
AUC 得分 (训练集): 0.856200


/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [69]:
xgb4 = XGBClassifier(
        learning_rate =0.01,
        n_estimators=400,
        max_depth=8,
        min_child_weight=5,
        gamma=0.2,
        subsample=0.7,
        colsample_bytree=0.8,
        reg_alpha=1e-05,
        objective= 'binary:logistic',
        nthread=1,
        scale_pos_weight=1,
        seed=27)
modelfit(xgb4, X, y)

(623, 21) (268, 21)

关于现在这个模型
准确率 : 0.7985
AUC 得分 (训练集): 0.878723


/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [76]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [79]:
svc1 = SVC(C=1.0, kernel='rbf', degree=3, gamma='auto',random_state=34)

In [80]:
#对reg_alpha用grid search寻找最合适的参数
param_test6 = {
    'C':[0.1, 0.5, 1, 1.5, 2]
}

gsearch1 = GridSearchCV(estimator = SVC(C=1.0, kernel='rbf', degree=3, gamma='auto',random_state=34),
                       param_grid = param_test6, scoring='roc_auc',n_jobs=1,iid=False, cv=5)
gsearch1.fit(X,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=34, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=False, n_jobs=1,
       param_grid={'C': [0.1, 0.5, 1, 1.5, 2]}, pre_dispatch='2*n_jobs',
       refit=True, scoring='roc_auc', verbose=0)

In [81]:
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: 0.85766, std: 0.01771, params: {'C': 0.1},
  mean: 0.84491, std: 0.02200, params: {'C': 0.5},
  mean: 0.84485, std: 0.02249, params: {'C': 1},
  mean: 0.85079, std: 0.02265, params: {'C': 1.5},
  mean: 0.85180, std: 0.02254, params: {'C': 2}],
 {'C': 0.1},
 0.857657962340029)

In [83]:
#对reg_alpha用grid search寻找最合适的参数
param_test6 = {
    'kernel':['linear', 'poly', 'rbf']
}

gsearch1 = GridSearchCV(estimator = SVC(C=0.1, kernel='rbf', degree=3, gamma='auto',random_state=34),
                       param_grid = param_test6, scoring='accuracy',n_jobs=1,iid=False, cv=5)
gsearch1.fit(X,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=34, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=False, n_jobs=1,
       param_grid={'kernel': ['linear', 'poly', 'rbf']},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [84]:
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: 0.77554, std: 0.02142, params: {'kernel': 'linear'},
  mean: 0.62628, std: 0.01093, params: {'kernel': 'poly'},
  mean: 0.78112, std: 0.01866, params: {'kernel': 'rbf'}],
 {'kernel': 'rbf'},
 0.7811222266078962)